In [1]:
import pandas as pd
import pandas_profiling as pp
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from matplotlib import pyplot
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.over_sampling import ADASYN

In [2]:
df = pd.read_csv('/home/junio/Desktop/Ju/mywaytohealth2020/jupyter/data/01.PATIENTS_VISITES-no_control_no_spurious_data-median-imputation_ADASYN.csv',sep=',')

In [3]:
df.shape

(2280, 23)

**Testing and evaluation of the regressor**

In [4]:
df = sklearn.utils.shuffle(df)
selected_fields = ['age', 'egfdrs001', 'nycturie_nb', 'score_depression', 'imc', 'perimetre_cervical', 'tour_de_hanches', 'padiast']
x = df[selected_fields]
y = df[['iah_class']]
#y = y.astype({"iah_class": int})
#x.isnull().sum()/len(df)*100
y.iah_class.value_counts()

1    576
2    569
0    569
3    566
Name: iah_class, dtype: int64

In [5]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x,y.values.ravel(),test_size=0.33,stratify=y.values.ravel())
n_iterations = 500

In [6]:
xgb=ExtraTreesClassifier(n_estimators=100, random_state=0)

In [7]:
xgb = ExtraTreesClassifier(n_estimators=100, random_state=0)

In [8]:
xgb.fit(x_train, y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=0, verbose=0,
                     warm_start=False)

In [9]:
y_pred=xgb.predict(x_test)
y_proba=xgb.predict_proba(x_test)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.ravel())
print(y_pred.shape)

(1527, 8)
(1527,)
(753, 8)
[2 3 0 3 2 0 3 1 3 3 2 3 2 0 0 0 2 1 1 3 3 3 3 2 0 1 3 0 3 2 2 2 2 0 3 1 0
 2 2 1 0 2 2 0 2 2 0 1 2 2 3 3 3 3 1 2 0 1 2 2 3 1 3 1 2 3 2 2 2 3 3 0 0 0
 3 2 2 0 2 3 2 1 0 2 1 3 1 0 2 3 2 0 2 2 0 3 1 3 1 0 3 3 1 3 3 1 0 2 3 3 2
 1 0 1 1 3 0 3 1 1 3 2 1 3 2 0 1 3 0 2 0 0 1 3 0 0 1 1 2 3 2 1 2 0 0 0 0 0
 3 2 2 1 1 1 2 0 0 0 2 1 2 1 3 3 0 0 1 0 2 0 2 0 1 3 1 0 2 0 3 1 1 3 1 0 3
 2 0 3 3 2 0 2 2 1 3 0 1 3 2 2 0 0 0 0 2 0 1 3 2 3 0 2 3 3 2 3 2 2 2 0 0 1
 1 3 1 0 1 0 1 0 1 2 3 1 3 2 1 1 0 0 1 2 1 1 1 1 2 2 0 1 3 0 1 3 1 3 3 1 3
 1 1 2 1 3 0 1 2 0 1 2 2 2 0 1 0 1 0 3 0 1 3 0 0 1 0 0 3 1 1 2 3 0 1 1 0 3
 0 1 1 3 3 0 3 1 1 2 1 3 3 2 2 1 3 1 2 2 2 0 3 1 1 2 3 3 3 1 2 1 0 1 2 2 1
 2 0 1 2 3 0 1 0 1 1 2 1 1 1 3 2 1 2 1 1 0 3 3 1 2 1 1 0 0 2 0 0 0 1 1 1 0
 0 0 2 3 2 3 2 1 0 1 3 3 1 0 2 3 1 3 2 3 2 1 0 3 3 0 3 3 0 0 0 1 1 1 1 2 2
 3 3 3 3 2 3 1 2 3 2 0 2 3 2 2 0 2 2 1 0 1 2 0 2 2 1 2 0 1 1 3 3 3 2 0 2 2
 0 1 2 1 3 2 0 0 0 3 3 0 2 3 2 3 2 3 3 3 2 1 2 2 0 0 2 3 0 0 3 3 1 2 0 1 

**100 rounds random sampling validation**

In [11]:
#The problem is imbalanced, the averageing MUST be 'macro' to reflect the minority class missclassifications
if True:
    selected_fields = ['age', 'egfdrs001', 'nycturie_nb', 'score_depression', 'imc', 'perimetre_cervical', 'tour_de_hanches', 'padiast']
    n_folds = 100
    df = pd.read_csv('/home/junio/Desktop/Ju/mywaytohealth2020/jupyter/data/01.PATIENTS_VISITES-no_control_no_spurious_data-median-imputation.csv',sep=',')
    comb_to_scores_MAP = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    print(selected_fields)
    for fold in range(n_folds):
        df = sklearn.utils.shuffle(df)
        x = df[selected_fields]
        y = df[['iah_class']]
        y = y.astype({"iah_class": int})
        x, y = ADASYN().fit_resample(x, y)
        x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x,y.values.ravel(),test_size=0.33,stratify=y.values.ravel())        
        #dfTemp = pd.DataFrame({'iah_class':y_train})
        #print(dfTemp.iah_class.value_counts())
        #print(dfTemp.shape)

        gnb = ExtraTreesClassifier(n_estimators=150)
        gnb.fit(x_train, y_train)
        y_pred = gnb.predict(x_test)
        y_prob = gnb.predict_proba(x_test)

        comb_to_scores_MAP[0] += f1_score(y_test, y_pred, average='micro')
        comb_to_scores_MAP[1] += f1_score(y_test, y_pred, average='macro')
        comb_to_scores_MAP[2] += f1_score(y_test, y_pred, average='weighted')
        comb_to_scores_MAP[3] += roc_auc_score(y_test, y_prob, average='macro', multi_class = 'ovr')
        comb_to_scores_MAP[4] += roc_auc_score(y_test, y_prob, average='macro', multi_class = 'ovo')        
        comb_to_scores_MAP[5] += roc_auc_score(y_test, y_prob, average='weighted', multi_class = 'ovr')
        comb_to_scores_MAP[6] += roc_auc_score(y_test, y_prob, average='weighted', multi_class = 'ovo')                
        comb_to_scores_MAP[7] += accuracy_score(y_test, y_pred)
        comb_to_scores_MAP[8] += precision_score(y_test, y_pred, average='micro',zero_division=0)
        comb_to_scores_MAP[9] += recall_score(y_test, y_pred, average='micro')
        comb_to_scores_MAP[10] += precision_score(y_test, y_pred, average='macro')
        comb_to_scores_MAP[11] += recall_score(y_test, y_pred, average='macro')

    comb_to_scores_MAP[0] = comb_to_scores_MAP[0] / n_folds
    comb_to_scores_MAP[1] = comb_to_scores_MAP[1] / n_folds
    comb_to_scores_MAP[2] = comb_to_scores_MAP[2] / n_folds
    comb_to_scores_MAP[3] = comb_to_scores_MAP[3] / n_folds
    comb_to_scores_MAP[4] = comb_to_scores_MAP[4] / n_folds
    comb_to_scores_MAP[5] = comb_to_scores_MAP[5] / n_folds
    comb_to_scores_MAP[6] = comb_to_scores_MAP[6] / n_folds
    comb_to_scores_MAP[7] = comb_to_scores_MAP[7] / n_folds
    comb_to_scores_MAP[8] = comb_to_scores_MAP[8] / n_folds
    comb_to_scores_MAP[9] = comb_to_scores_MAP[9] / n_folds
    comb_to_scores_MAP[10] = comb_to_scores_MAP[10] / n_folds
    comb_to_scores_MAP[11] = comb_to_scores_MAP[11] / n_folds
    print('F1-Micro',comb_to_scores_MAP[0])
    print('F1-Macro',comb_to_scores_MAP[1])
    print('F1-Weighted',comb_to_scores_MAP[2])
    print('AUC-ROC-Macro-ovr',comb_to_scores_MAP[3])
    print('AUC-ROC-Macro-ovo',comb_to_scores_MAP[4])
    print('AUC-ROC-Weighted-ovr',comb_to_scores_MAP[5],"=> use ovr instead of ovo")
    print('AUC-ROC-Weighted-ovo',comb_to_scores_MAP[6])
    print('Accuracy',comb_to_scores_MAP[7])
    print('Precision-micro',comb_to_scores_MAP[8])
    print('Recall-micro',comb_to_scores_MAP[9])
    print('Precision-macro',comb_to_scores_MAP[10])
    print('Recall-macro',comb_to_scores_MAP[11])


    target_names = ['normal', 'mild', 'moderate', 'severe']
    print(classification_report(y_test, y_pred, labels=[0, 1, 2, 3], target_names=target_names))

    #https://statinfer.com/204-4-2-calculating-sensitivity-and-specificity-in-python/
    cm = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix : \n', cm)

    total=sum(sum(cm))
    #####from confusion matrix calculate accuracy
    accuracy=(cm[0,0]+cm[1,1]+cm[2,2]+cm[3,3])/total
    print ('Accuracy : ', accuracy)

    sensitivity=[0,0,0,0]
    sensitivity[0] = cm[0,0]/(cm[0,0]+cm[0,1]+cm[0,2]+cm[0,3])
    print('Sensitivity class 0: ', sensitivity[0])
    sensitivity[1] = cm[1,1]/(cm[1,0]+cm[1,1]+cm[1,2]+cm[1,3])
    print('Sensitivity class 1: ', sensitivity[1])
    sensitivity[2] = cm[2,2]/(cm[2,0]+cm[2,1]+cm[2,2]+cm[2,3])
    print('Sensitivity class 2: ', sensitivity[2])
    sensitivity[3] = cm[3,3]/(cm[3,0]+cm[3,1]+cm[3,2]+cm[3,3])
    print('Sensitivity class 3: ', sensitivity[3])
    print('Average sensitivity:',np.mean(sensitivity))
    print()

    specificity = [0,0,0,0]
    specificity[0] = (cm[1,1]+cm[2,2]+cm[3,3])/((cm[1,1]+cm[2,2]+cm[3,3]) + (cm[1,0]+cm[2,0]+cm[3,0]))
    print('Specificity class 0: ', specificity[0])
    specificity[1] = (cm[0,0]+cm[2,2]+cm[3,3])/((cm[0,0]+cm[2,2]+cm[3,3]) + (cm[0,1]+cm[2,1]+cm[3,1]))
    print('Specificity class 1: ', specificity[1])
    specificity[2] = (cm[0,0]+cm[1,1]+cm[3,3])/((cm[0,0]+cm[1,1]+cm[3,3]) + (cm[0,2]+cm[1,2]+cm[3,2]))
    print('Specificity class 2: ', specificity[2])
    specificity[3] = (cm[0,0]+cm[1,1]+cm[2,2])/((cm[0,0]+cm[1,1]+cm[2,2]) + (cm[0,3]+cm[1,3]+cm[2,3]))
    print('Specificity class 3: ', specificity[3])
    print('Average specificity:',np.mean(specificity))